# Distributed Training On NVIDIA DGX Station A100

In [6]:
#import paskages 
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [7]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="4"

In [11]:
tf.test.is_built_with_cuda()

False

In [12]:
tf.config.experimental.list_physical_devices()


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [13]:
tf.config.experimental.list_logical_devices()


[LogicalDevice(name='/device:CPU:0', device_type='CPU')]

In [14]:
tf.distribute.MirroredStrategy().num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


1

### Load the dataset


In [19]:
(X_train , y_train) ,(X_test , y_test) = tf.keras.datasets.cifar10.load_data()

In [20]:
X_train.shape

(50000, 32, 32, 3)

In [21]:
classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

In [22]:
y_train[:5]

array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

In [23]:
X_train_scaled =X_train / 255
X_test_scaled = X_test / 255

In [52]:
y_train_categorical = keras.utils.to_categorical(
                                y_train , num_classes=10 ,dtype='float32'
                                                        )
y_test_categorical = keras.utils.to_categorical(
                                y_test , num_classes=10 ,dtype='float32'
                                                        )


In [25]:
y_train_categorical

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [51]:
def get_model():
    model= tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(32,32,3)),
        tf.keras.layers.Dense(3000 , activation='relu'),
        tf.keras.layers.Dense(1000 , activation='relu'),
        tf.keras.layers.Dense(10 , activation='sigmoid')
    ])
    
    
    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [54]:
train_tf_dataset = tf.data.Dataset.from_tensor_slices((X_train_scaled, y_train_categorical))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((X_test_scaled, y_test_categorical))

In [55]:
type(train_tf_dataset)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [57]:
strategy= tf.distribute.MirroredStrategy()
strategy.num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


1

In [60]:
BATCH_SIZE_PER_REPLICA = 250
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
BATCH_SIZE

250

In [63]:
train_dataset=train_tf_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_tf_dataset.batch(BATCH_SIZE)

In [73]:
%%timeit -n1 -r1 
with strategy.scope():
    gpu_model = get_model()
    gpu_model.fit(train_dataset , epochs=5)

Epoch 1/5
200/200 [==============================] - 26s 125ms/step - loss: 1.9831 - accuracy: 0.2978
Epoch 2/5
200/200 [==============================] - 27s 135ms/step - loss: 1.8187 - accuracy: 0.3649
Epoch 3/5
200/200 [==============================] - 30s 149ms/step - loss: 1.7564 - accuracy: 0.3871
Epoch 4/5
200/200 [==============================] - 30s 152ms/step - loss: 1.7133 - accuracy: 0.4031
Epoch 5/5
200/200 [==============================] - 26s 129ms/step - loss: 1.6792 - accuracy: 0.4152
2min 22s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [74]:
%%timeit -n1 -r1 
with tf.device("/CPU:0"):
    cpu_model=get_model()
    cpu_model.fit(train_dataset , epochs=5)

Epoch 1/5
200/200 [==============================] - 25s 122ms/step - loss: 1.9827 - accuracy: 0.2979
Epoch 2/5
200/200 [==============================] - 28s 139ms/step - loss: 1.8166 - accuracy: 0.3651
Epoch 3/5
200/200 [==============================] - 31s 157ms/step - loss: 1.7561 - accuracy: 0.3876
Epoch 4/5
200/200 [==============================] - 29s 145ms/step - loss: 1.7137 - accuracy: 0.4048
Epoch 5/5
200/200 [==============================] - 29s 147ms/step - loss: 1.6797 - accuracy: 0.4176
2min 22s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
